In [3]:
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

In [15]:
from src.hatefuldata import HatefulMemesDataset
from src.multimodel import HatefulMemesModel, LanguageAndVisionConcat

In [49]:
df = pd.read_json('data/train.jsonl', lines=True)

In [50]:
df_sorted = df.copy().sort_values('id').reset_index().drop('index', axis=1)

In [51]:
df_sorted['text']

0       when you're feeling horny asf but your habibi ...
1                        meanwhile at the isis strip club
2       when your human says "who' s a good girl?" and...
3       and that was the last nativity play my son was...
4           you can't be racist if there is no other race
                              ...                        
8495                                      shut up ontario
8496    400 years of slavery rape and murder but now s...
8497    this dude wants to use the same restroom as yo...
8498                child in poverty with empty water jug
8499                 african kids dancing and celebrating
Name: text, Length: 8500, dtype: object

In [52]:
df_sorted.label.value_counts()

0    5450
1    3050
Name: label, dtype: int64

In [53]:
#Compute typical meme 'sentence' length & describe
df_sorted.text.map(
    lambda text: len(text.split(" "))
).describe()

count    8500.000000
mean       11.748706
std         6.877880
min         1.000000
25%         7.000000
50%        10.000000
75%        15.000000
max        70.000000
Name: text, dtype: float64

In [54]:
#Look at image sizes
images = [
    Image.open(
        'data/' + str(df_sorted.loc[i, "img"])
    ).convert("RGB")
    for i in range(5)
]

for image in images:
    print(image.size)

(550, 366)
(712, 800)
(525, 800)
(550, 411)
(277, 399)


In [19]:
import torch
import torchvision

In [20]:
# define a callable image_transform with Compose
image_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(224, 224)),
        torchvision.transforms.ToTensor()
    ]
)

# convert the images and prepare for visualization.
tensor_img = torch.stack(
    [image_transform(image) for image in images]
)
grid = torchvision.utils.make_grid(tensor_img)

# plot
plt.rcParams["figure.figsize"] = (20, 5)
plt.axis('off')
_ = plt.imshow(grid.permute(1, 2, 0))

NameError: name 'images' is not defined

In [10]:
hparams = {
    
    # Required hparams
    "train_path": 'data/train.jsonl',
    "dev_path": 'data/dev.jsonl',
    "img_dir": 'data/',
    
    # Optional hparams
    "embedding_dim": 150,
    "language_feature_dim": 300,
    "vision_feature_dim": 300,
    "fusion_output_size": 256,
    "output_path": "model-outputs",
    "dev_limit": None,
    "lr": 0.00005,
    "max_epochs": 10,
    "n_gpu": 0,
    "batch_size": 4,
    # allows us to "simulate" having larger batches 
    "accumulate_grad_batches": 16,
    "early_stop_patience": 3,
}

hateful_memes_model = HatefulMemesModel(hparams=hparams)

FileNotFoundError: 

In [ ]:
hateful_memes_model.fit()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type                    | Params
--------------------------------------------------
0 | model | LanguageAndVisionConcat | 58 M  


In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4806160487254813576, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6420989224621059723
 physical_device_desc: "device: XLA_CPU device"]

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

[]